In [23]:
import os
from transformers import GPT2LMHeadModel, GPT2Tokenizer, BertTokenizer, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling
import torch

In [24]:
# 1. 加载所有txt文件内容
def load_all_texts(data_dir="./data"):
    texts = []
    for fname in os.listdir(data_dir):
        if fname.endswith(".txt"):
            path = os.path.join(data_dir, fname)
            try:
                with open(path, "r", encoding="utf-8") as f:
                    texts.append(f.read())
            except UnicodeDecodeError:
                with open(path, "r", encoding="gbk", errors="ignore") as f:
                    texts.append(f.read())
    return "\n".join(texts)

In [25]:
# 2. 合并成一个大的corpus文件
def merge_texts(data_dir="./data", output_file="merged_corpus.txt"):
    texts = load_all_texts(data_dir)
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(texts)

In [26]:
# 3. 加载Dataset
def load_dataset(file_path, tokenizer, block_size=128):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )

In [27]:
# 4. 文本生成函数
def generate_text(model, tokenizer, prompt="Hello", max_length=100):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
    output_ids = model.generate(input_ids, max_length=max_length, do_sample=True, top_k=50, top_p=0.95)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

In [43]:
# ============== 开始执行 ==============

# 步骤1：准备数据
merge_texts("./data", "merged_corpus.txt")

# 步骤2：加载预训练模型
tokenizer = BertTokenizer.from_pretrained("uer/gpt2-chinese-cluecorpussmall")
model = GPT2LMHeadModel.from_pretrained("uer/gpt2-chinese-cluecorpussmall")
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# 步骤3：测试微调前生成效果
print("\n=== 微调前测试生成 ===")
print(generate_text(model, tokenizer, prompt="在黄沙莽莽的回疆大漠之上", max_length=150))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



=== 微调前测试生成 ===
在 黄 沙 莽 莽 的 回 疆 大 漠 之 上 当 年 的 民 族 情 怀 都 是 什 么 样 的 ？ 在 黄 沙 莽 莽 的 回 疆 大 漠 之 上 ， 每 年 大 量 的 回 民 涌 入 ， 当 然 每 年 会 有 很 多 人 回 汉 大 战 的 悲 壮 事 件 ， 比 如 ： 青 藏 铁 路 开 通 红 军 翻 越 乌 兰 布 统 雪 山 西 部 地 区 被 英 国 侵 略 等 等 等 。 老 百 姓 都 很 开 心 这 种 问 题 不 在 黄 沙 莽 莽 的 回 疆 大 漠 之 上 。 我 不 想 让 你 们 因 为 我 的 无 知 而 蒙 受


In [44]:
# 步骤4：微调训练
train_dataset = load_dataset("merged_corpus.txt", tokenizer)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
    prediction_loss_only=True,
    logging_steps=100,
    logging_dir='./logs',
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

trainer.train()

d:\documents\COURSE\DLNLP\DL-NLP-2025-LSX\hw4\.venv\lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
100,3.756700


KeyboardInterrupt: 

In [46]:
# 步骤5：测试微调后生成效果
print("\n=== 微调后测试生成 ===")
print(generate_text(model, tokenizer, prompt="在黄沙莽莽的回疆大漠之上，", max_length=200))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



=== 微调后测试生成 ===
在 黄 沙 莽 莽 的 回 疆 大 漠 之 上 ， 能 想 起 的 只 剩 下 这 几 日 。 此 时 ， 天 下 只 剩 黄 沙 ， 不 会 流 下 一 滩 水 ， 哪 里 来 的 这 样 一 口 好 水 ？ 说 起 黄 沙 莽 莽 ， 黄 沙 还 有 几 日 就 要 退 了 。 那 些 人 已 下 马 ， 只 见 在 沙 漠 里 走 了 好 几 个 人 ， 当 然 没 有 走 。 沙 漠 里 面 还 有 三 个 敌 人 ， 当 日 晚 上 ， 三 ， 又 有 一 个 人 过 来 亦 蒙 ， 你 他 ？
